In [2]:
import re

In [26]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [27]:
def print_if_verbose(text, verbose):
    if verbose:
        print(text)

In [29]:
def lowercase_and_tokenize(s):
    return [t.lower() for t in word_tokenize(s)]


def get_first_token(s):
    tokenized = word_tokenize(s)
    if len(tokenized) == 0:
        return None
    else:
        return tokenized[0].lower()

In [30]:
def find_nested_relation(
    relation_type, relation, cur_depth=1, max_depth=3
):
    if relation is None or cur_depth > max_depth:
        return None, None
    elif relation_type == relation.type:
        return relation, cur_depth
    else:
        left_result, depth = find_nested_relation(
            relation_type, relation.left_child, cur_depth + 1
        )
        if left_result is None:
            return find_nested_relation(
                relation_type, relation.right_child, cur_depth + 1
            )
        else:
            return left_result, depth

In [31]:
# Testing

# %run ./relation_extraction.ipynb

# relation = Relation(
#     "Explanation", 
#     1, 
#     None,
#     Relation(
#         "Background",
#         2,
#         None,
#         None,
#         None
#     ),
#     None
# )

# print(find_nested_relation("Background", None))

# print(find_nested_relation("Explanation", relation))

# print(find_nested_relation("Background", relation))

# relation = Relation(
#     "Explanation", 
#     1, 
#     None,
#     Relation(
#         "Background",
#         2,
#         None,
#         None,
#         None
#     ),
#     Relation(
#         "Background",
#         3,
#         None,
#         Relation(
#             "Elaboration",
#             4,
#             None,
#             None,
#             None
#         ),
#         None
#     )
# )

# print(find_nested_relation("Background", relation))

# print(find_nested_relation("Elaboration", relation))

In [ ]:
with open("aux/connectives.txt", "rt") as f:
    connectives = set(
        [line.strip() for line in f.readlines()]
    )

In [3]:
punctuation = set([".", ",", "!", "?", ";"])

In [4]:
def remove_trailing_punctuation(string):
    if len(string) == 0 or string[-1] not in punctuation:
        return string
    else:
        return string[:-1]

In [5]:
def fix_spacing(string):
    if len(string) > 0 and string[-1] in punctuation:
        return string[:-1].strip() + string[-1]
    else:
        return string

In [41]:
def remove_leading_punctuation(string):
    if len(string) == 0 or string[0] not in punctuation:
        return string
    else:
        return string[1:].strip()

In [42]:
def lowercase_first_letter(string):
    if "".join(word_tokenize(string)[:1]) != "I":
        return string[:1].lower() + string[1:]
    else:
        return string


def uppercase_first_letter(string):
    return string[:1].upper() + string[1:]

In [43]:
def trim_connective(string):
    lower = string.lower()
    for c in connectives:
        if lower[:len(c)] == c:
            if (
                c != "last" 
                or (len(lower) > len(c) and lower[len(c)] == ",")
            ):
                return remove_leading_punctuation(
                    string[len(c):]
                )
            else:
                return string
    return string

In [62]:
def get_relation_type(relation):
    if relation is None:
        return "-"
    else:
        return relation.type

In [4]:
def remove_extra_space(s):
    return " ".join(s.split())


def contains_any_of(s, s_array):
    for other_s in s_array:
        if other_s in s:
            return True
    return False